In [ ]:
import json

参照MSR-VTT的标注构造json文件

In [ ]:
anns_txt_file = './AllVideoDescriptions.txt'
video_ids = []
anns_json = {
    'info': {
        'contributor': 'Microsoft', 
        'description': 'MSVD dataset. A total of 1970 videos, where 1200 for training, 100 for validate, and 670 for testing.', 
        'year': 2011
    },
    'videos': [],
    'sentences': []
}

anns_json_coco = {
    'info': {
        'contributor': 'Microsoft', 
        'description': 'MSVD dataset. A total of 1970 videos, where 1200 for training, 100 for validate, and 670 for testing.', 
        'year': 2011
    },
    'licenses': ['MSVD video captioning dataset'],
    'images': [],
    'annotations': []
}

sen_id = 0
vid_id = 0
with open(anns_txt_file, 'r') as f:
    for _line in f.readlines():
        _line = _line.strip()
        if _line.startswith('#') or len(_line) == 0:
            continue
        _infos = _line.split(' ')
        _video_id = _infos[0]
        _caption = " ".join(_infos[1:])
        anns_json['sentences'].append({
            'video_id': _video_id,
            'caption': _caption,
            'sen_id': sen_id
        })
        anns_json_coco['annotations'].append({
            'image_id': _video_id,     # 视频id
            'caption': _caption,       # 视频描述语句
            'id': sen_id               # 描述语句id
        })
        sen_id += 1
        
        if _video_id not in video_ids:
            video_ids.append(_video_id)
            if len(video_ids) <= 1200:
                split = 'train'
            elif len(video_ids) <= 1300:
                split = 'validate'
            else:
                split = 'test'
            anns_json['videos'].append({
                'video_id': _video_id,
                'split': split,
                'id': vid_id
            })
            anns_json_coco['images'].append({
                'file_name': _video_id,     # 视频文件名
                'split': split,             # 数据集划分类别 train / validate / test
                'id': _video_id             # 视频id，应与‘annotations’字段下的image_id一致
            })
            vid_id += 1
        # print('id: %s, caption: %s' % (_video_id, _caption))
        
print(len(video_ids))
json.dump(anns_json, open('./MSVD_annotations.json', 'w'))
json.dump(anns_json_coco, open('./MSVD_annotations_coco.json', 'w'))

In [ ]:
import copy

# 分别存储val数据和test数据
val_video_ids = [_data['video_id'] for _data in anns_json['videos']  if _data['split'] == 'validate']
test_video_ids = [_data['video_id'] for _data in anns_json['videos']  if _data['split'] == 'test']

# MSR-VTT格式
anns_val_json = {
    'info': copy.deepcopy(anns_json['info']),
    'videos': [],
    'sentences': []
}
anns_test_json = {
    'info': copy.deepcopy(anns_json['info']),
    'videos': [],
    'sentences': []
}

for _data in anns_json['videos']:
    if _data['split'] == 'validate':
        anns_val_json['videos'].append(_data)
    elif _data['split'] == 'test':
        anns_test_json['videos'].append(_data)
    else:
        continue
        
for _data in anns_json['sentences']:
    if _data['video_id'] in val_video_ids:
        anns_val_json['sentences'].append(_data)
    elif _data['video_id'] in test_video_ids:
        anns_test_json['sentences'].append(_data)
    else:
        continue
        
json.dump(anns_val_json, open('MSVD_val100.json', 'w'))
json.dump(anns_test_json, open('MSVD_test670.json', 'w'))


# COCO格式
anns_val_json_coco = {
    'info': copy.deepcopy(anns_json_coco['info']),
    'licenses': copy.deepcopy(anns_json_coco['licenses']),
    'images': [],
    'annotations': []
}
anns_test_json_coco = {
    'info': copy.deepcopy(anns_json_coco['info']),
    'licenses': copy.deepcopy(anns_json_coco['licenses']),
    'images': [],
    'annotations': []
}

for _data in anns_json_coco['images']:
    if _data['split'] == 'validate':
        anns_val_json_coco['images'].append(_data)
    elif _data['split'] == 'test':
        anns_test_json_coco['images'].append(_data)
    else:
        continue
        
for _data in anns_json_coco['annotations']:
    if _data['image_id'] in val_video_ids:
        anns_val_json_coco['annotations'].append(_data)
    elif _data['image_id'] in test_video_ids:
        anns_test_json_coco['annotations'].append(_data)
    else:
        continue
        
json.dump(anns_val_json_coco, open('MSVD_val100_coco.json', 'w'))
json.dump(anns_test_json_coco, open('MSVD_test670_coco.json', 'w'))

In [ ]:
# 构造一个测试数据
pred_example = []
_video_ids = []
with open(anns_txt_file, 'r') as f:
    for _line in f.readlines():
        _line = _line.strip()
        if _line.startswith('#') or len(_line) == 0:
            continue
        _infos = _line.split(' ')
        _video_id = _infos[0][1:]
        _caption = " ".join(_infos[1:])
        if _video_id not in _video_ids:
            _video_ids.append(_video_id)
            pred_example.append({
                'image_id': _video_id,
                'caption': _caption
            })
            
json.dump(pred_example, open('pred_example.json', 'w'))